In [1]:
import re
import time
import json
import numpy as np
import pandas as pd

# Reading Mat File
import scipy.io

# To find Great Circle Distance
from cartopy import geodesic
import shapely.geometry as sgeom

In [2]:
#0-80km, 0-100km, and 200-400km
#col_name = ['0-50','50-100','100-150','150-200','200-250','250-300','300-350','350-400']
#col_radius = [(0,50),(50,100),(100,150),(150,200),(200,250),(250,300),(300,350),(350,400)]
col_name = ['0-80','0-100','200-400']
col_radius = [(0,80),(0,100),(200,400)]
map_radius_to_name = {}
for i in range(len(col_name)):
    map_radius_to_name[col_radius[i]] = col_name[i]

In [3]:
imgFreq = {}
fList = ['19H','19V','19','22V','37V','37H','37','91H','91V','91','150H','183_1H','183_3H','183_7H']
rList = ['ATL','CPAC','EPAC','IO','SHEM','WPAC']
for r in rList:
    imgFreq[r] = {}
    for f in fList:
        if 'H' not in f and 'V' not in f:
            imgFreq[r][f] = pd.read_csv("..\\ImagesPerFreq\\"+r+"_"+f+'PCT.csv')
        else:
            imgFreq[r][f] = pd.read_csv("..\\ImagesPerFreq\\"+r+"_"+f+'.csv')
        for rad in col_name:
            imgFreq[r][f][rad] = np.nan

In [4]:
imgFreq['ATL']['19']

,FileName,CenLon,CenLat,Pressure,Wind,Area,T_No,0-50,50-100,100-150,150-200,200-250,250-300,300-350,350-400,0-80,0-100,200-400
0,..\..\MyCreatedData\5\ATL\29\F16\19\20051122T2...,-40.820101,30.037247,984,45,0.731821,3.0,280.577962,281.875653,283.998764,285.439172,286.407243,287.136546,287.556750,287.812071,NaN,NaN,NaN
1,..\..\MyCreatedData\5\ATL\29\F16\19\20051123T1...,-41.221494,27.498088,983,50,0.990081,3.0,283.010836,283.102151,284.303013,284.894224,285.627855,286.314580,287.155166,288.082327,NaN,NaN,NaN
2,..\..\MyCreatedData\5\ATL\29\F16\19\20051123T2...,-40.319371,25.653805,982,55,0.997904,3.5,286.988969,285.121665,284.548524,285.720195,287.380519,288.160216,289.081811,289.890177,NaN,NaN,NaN
3,..\..\MyCreatedData\5\ATL\29\F16\19\20051124T1...,-39.045968,24.814455,980,60,0.996655,3.5,283.728767,283.453010,284.355134,284.892815,286.358307,287.842468,288.934561,289.450696,NaN,NaN,NaN
4,..\..\MyCreatedData\5\ATL\29\F16\19\20051125T1...,-39.561312,23.379201,982,55,0.997899,3.5,285.336194,284.131648,284.881809,286.260307,287.985572,289.228951,289.891106,290.335442,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3860,..\..\MyCreatedData\16\ATL\15\F18\19\20161017T...,-42.846875,42.632057,962,65,0.998024,4.0,284.026970,281.497738,280.482932,280.937620,281.888724,282.656251,283.282366,283.330003,NaN,NaN,NaN
3861,..\..\MyCreatedData\16\ATL\15\F18\19\20161018T...,-37.058067,47.672898,969,55,0.291986,3.5,NaN,NaN,279.023002,278.468717,278.256106,278.495983,278.263101,279.237673,NaN,NaN,NaN
3862,..\..\MyCreatedData\16\ATL\69\F16\19\20161102T...,-46.052121,17.591951,1009,20,0.635043,0.0,294.599187,294.716096,294.919914,295.508521,296.161805,296.151446,296.078709,295.261321,NaN,NaN,NaN
3863,..\..\MyCreatedData\16\ATL\69\F17\19\20161102T...,-46.543180,17.853565,1009,20,0.940508,0.0,291.043348,291.774440,292.220666,292.619892,292.896426,292.740908,292.479544,292.805927,NaN,NaN,NaN


In [5]:
valid_matfile_and_best_track = json.load( open( "valid_matfile_and_best_track.json" ) )

In [6]:
class CDF_of_Tbs:
    global map_radius_to_name
    global imgFreq
    def __init__(self, matpath):
        self.mMatPath = matpath
        
        self.__mFreq_to_swath = {}
        self.__mFreq_to_swath['19']  = [0,[2.400, 1.400]] 
        self.__mFreq_to_swath['19V'] = [0,0]
        self.__mFreq_to_swath['19H'] = [0,1]
        self.__mFreq_to_swath['22V'] = [0,2]

        self.__mFreq_to_swath['37']  = [1,[2.150, 1.150]]
        self.__mFreq_to_swath['37V'] = [1,0]
        self.__mFreq_to_swath['37H'] = [1,1]

        self.__mFreq_to_swath['91']  = [3,[1.751, 0.751]]
        self.__mFreq_to_swath['91V'] = [3,0] 
        self.__mFreq_to_swath['91H'] = [3,1]

        self.__mFreq_to_swath['150H'] = [2,0]
        self.__mFreq_to_swath['183_1H'] = [2,1]
        self.__mFreq_to_swath['183_3H'] = [2,2]
        self.__mFreq_to_swath['183_7H'] = [2,3]
     
    def PCT_Function(self,v,h,val):
        return val[0]*v - val[1]*h
    
    def ReadMatFile(self,region,matFile,freq_data):
        try:
            mat = scipy.io.loadmat(self.mMatPath+matFile)
        except   Exception as e:
            print(e)
            msg = "Error Reading File: " + str( self.mMatPath+matFile ) 
            print(msg)
            #self.__mLog.error( msg )
       
        swaths = mat["passData"][0][0]
        
        for freq,cenLon,cenLat,t_no,index_no in freq_data:
            swathList = self.__mFreq_to_swath[freq]
            
            swath_data = swaths[ swathList[0] ]
            lat = swath_data[0][0][1]
            lon = swath_data[0][0][2]
            channel = swath_data[0][0][3]
            # Calculating PCT values for specific frequencies
            if freq == '19' or freq=='37' or freq=='91':
                tbs = self.PCT_Function( channel[0], channel[1], swathList[1] )
            else:
                tbs = channel[swathList[1]]
            
            pre_indices = []
            for (r1,r2),col_name in map_radius_to_name.items():#[50,100,150,200,250,300,350,400]:
                
                def GetIndices(radius):                
                    circle= geodesic.Geodesic().circle(cenLon, cenLat, radius*1000)
                    poly = sgeom.Polygon(circle)

                    #                                 min Lon                max Lon
                    result = np.where( (lon>=poly.bounds[0]) & (lon<=poly.bounds[2]) )
                    lon_set = set(zip(result[0], result[1]))
                    #                                 min Lat                max Lat
                    result = np.where( (lat>=poly.bounds[1]) & (lat<=poly.bounds[3]) )
                    lat_set = set(zip(result[0], result[1]))
                    
                    return lon_set.intersection(lat_set)
                
                curr_indices = list(GetIndices(r1) ^ GetIndices(r2))
                
                tot_tbs = 0
                for i,j in curr_indices:
                    tot_tbs+=tbs[i,j]
                try:
                    avg_tbs = tot_tbs/len(curr_indices)
                except ZeroDivisionError:
                    #print("ZeroDivision for: ",region,freq,matFile[:25],r1,r2)
                    continue
                
                col_index = imgFreq[region][freq].columns.get_loc(col_name)
                imgFreq[region][freq].iloc[index_no,col_index] = avg_tbs

In [7]:
s = time.time()
for region,v1 in valid_matfile_and_best_track.items():
    for year,v2 in v1.items():
        print(region,year)
        for stormNo,stormDict in v2.items():
            for f_,matFiles in stormDict.items():
                rootDirOfMatFile = '..\\..\\CurrentData\\Passtimes_and_1C_Data\\raw_data\\'+year+"\\"+region+"\\"+stormNo+"\\passtimes\\SSMIS\\"+f_+"\\"
                cdf_tbs = CDF_of_Tbs(rootDirOfMatFile)
                for filename,freq_data in matFiles.items():
                    cdf_tbs.ReadMatFile(region, filename[:25]+".mat",freq_data)
print("Total Time taken: "+str( (time.time()-s)/60 )+" mins" )

ATL 5
ATL 12
ATL 6
ATL 17
ATL 7
ATL 8
ATL 9
ATL 10
ATL 18
ATL 13
ATL 11
ATL 14
ATL 15
ATL 19
ATL 16
CPAC 6
CPAC 9
CPAC 18
CPAC 13
CPAC 14
CPAC 15
CPAC 19
CPAC 16
EPAC 12
EPAC 6
EPAC 17
EPAC 7
EPAC 8
EPAC 9
EPAC 10
EPAC 18
EPAC 13
EPAC 11
EPAC 14
EPAC 15
EPAC 19
EPAC 16
IO 5
IO 12
IO 17
IO 7
IO 8
IO 9
IO 10
IO 18
IO 13
IO 11
IO 14
IO 15
IO 19
IO 16
SHEM 12
SHEM 6
SHEM 17
SHEM 7
SHEM 8
SHEM 9
SHEM 10
SHEM 18
SHEM 13
SHEM 11
SHEM 14
SHEM 15
SHEM 19
SHEM 16
WPAC 12
WPAC 6
WPAC 17
WPAC 7
WPAC 8
WPAC 9
WPAC 10
WPAC 18
WPAC 13
WPAC 11
WPAC 14
WPAC 15
WPAC 19
WPAC 16
Total Time taken: 67.3064949274063 mins


In [9]:
for r in rList:
    for f in fList:
        if 'H' not in f and 'V' not in f:
            imgFreq[r][f].to_csv("..\\ImagesPerFreq_old\\"+r+"_"+f+"PCT.csv",index=False)
        else:
            imgFreq[r][f].to_csv("..\\ImagesPerFreq_old\\"+r+"_"+f+".csv",index=False)

In [ ]:
imgFreq['ATL']['19H']

In [ ]:
50
149
317
516
634
849
1002
1244

In [ ]:
land_shp_fname = shpreader.natural_earth(resolution='50m',
                                       category='physical', name='land')

land_geom = unary_union(list(shpreader.Reader(land_shp_fname).geometries()))
land = prep(land_geom)

def IsLand(lon,lat):
    global land
    for x in lon:
        for y in lat:
            if land.contains(sgeom.Point(x,y)):
                return True
    return False

def GetSerialDate( normal_date ):
    return mpldate.date2num(normal_date)

def GetSerialDateFromString( str_name ):
    return mpldate.datestr2num(str_name) 

def MakeDir(path):
    if os.path.isdir(path) == False:
        os.mkdir(path)

In [ ]:
class MatReader():   
    def __init__(self,matpath):
        # Will be useful in Child Class
        self._mErrors = {}
        self._mMatPath = "..\\"+matpath
             
    def ReadMatFile(self, matFile):
        self._mErrors[0] = ["Invalid Lat and Lon values     : ",[]]
        self._mErrors[1] = ["Invalid shape of Frequency     : ",[]]
        self._mErrors[2] = ["Invalid Brightness Temperature : ",[]]
        self._mErrors[3] = ["Land found in smaller region   : ",[]]
        
        try:
            mat = scipy.io.loadmat(self._mMatPath+matFile)
        except:
            msg = "Error Reading File: " + str( self._mMatPath+matFile ) 
            print( msg )
            return
        
        swaths = mat["passData"][0][0]
        return swaths

In [ ]:
valid_matfile_and_best_track = json.load( open( "..\\valid_matfiles_and_best_track.json" ) )

In [ ]:
class CDF_Of_Tbs:
    global tbsVal
    def __init__(self, matpath):
        self.mMatPath = matpath
        
        self.__mFreq_to_swath = {}
        self.__mFreq_to_swath['19']  = [0,[2.400, 1.400]] 
        self.__mFreq_to_swath['19V'] = [0,0]
        self.__mFreq_to_swath['19H'] = [0,1]
        self.__mFreq_to_swath['22V'] = [0,2]

        self.__mFreq_to_swath['37']  = [1,[2.150, 1.150]]
        self.__mFreq_to_swath['37V'] = [1,0]
        self.__mFreq_to_swath['37H'] = [1,1]

        self.__mFreq_to_swath['91']  = [3,[1.751, 0.751]]
        self.__mFreq_to_swath['91V'] = [3,0] 
        self.__mFreq_to_swath['91H'] = [3,1]

        self.__mFreq_to_swath['150H'] = [2,0]
        self.__mFreq_to_swath['183_1H'] = [2,1]
        self.__mFreq_to_swath['183_3H'] = [2,2]
        self.__mFreq_to_swath['183_7H'] = [2,3]
     
    def PCT_Function(self,v,h,val):
        return val[0]*v - val[1]*h
    
    def ReadMatFile(self,region,matFile,valid_freq):
        try:
            mat = scipy.io.loadmat(self.mMatPath+matFile)
        except:
            msg = "Error Reading File: " + str( self.mMatPath+matFile ) 
            print(msg)
            #self.__mLog.error( msg )
       
        swaths = mat["passData"][0][0]
        
        for freq in valid_freq:
            swathList = self.__mFreq_to_swath[freq]
            
            swath_data = swaths[ swathList[0] ]
            lat = swath_data[0][0][1]
            lon = swath_data[0][0][2]
            channel = swath_data[0][0][3]
            
             # Calculating PCT values for specific frequencies
            if freq == '19' or freq=='37' or freq=='91':
                tbs = self.PCT_Function( channel[0], channel[1], swathList[1] )
            else:
                tbs = channel[swathList[1]]
            
            if np.any(tbs<320) and np.any(tbs>0):
                tbsVal[region][freq][0].append(np.amin(tbs))
                tbsVal[region][freq][1].append(np.amax(tbs))